In [3]:
from sklearn.preprocessing import FunctionTransformer

from reskit.norms import binar_norm, wbysqdist
from reskit.norms import spectral_norm

from reskit.features import degrees,  pagerank

from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier 
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from reskit.core import Transformer, Pipeliner

def orig(x):
    return x

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Функция считывания данных

In [4]:
def get_autism(path_to_read='Data/dti/', distances=True):
    def get_autism_distances(loc_name):
        with open(loc_name, 'r') as f:
            read_data = f.readlines()

        read_data = pd.DataFrame(
            np.array([np.array(item[:-1].split()).astype(int) for item in read_data]))

        return read_data

    def get_distance_matrix(coords):
        if type(coords) == pd.core.frame.DataFrame:
            coords = coords.values
        elif type(coords) != np.ndarray:
            print('Provide either pandas df or numpy array!')
            return -1

        shape = len(coords)
        dist_matrix = np.zeros((shape, shape))
        del shape
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                dist_matrix[i, j] = np.linalg.norm(coords[i, :] - coords[j, :])
                dist_matrix[j, i] = dist_matrix[i, j]
        return dist_matrix

    target_vector = []  # this will be a target vector (diagnosis)
    matrices = []  # this will be a list of connectomes
    all_files = sorted(os.listdir(path_to_read))
    matrix_files = [
        item for item in all_files if 'DTI_connectivity' in item and 'All' not in item]
    distance_files = [
        item for item in all_files if 'DTI_region_xyz_centers' in item and 'All' not in item]

    # for each file in a sorted (!) list of files:
    for filename in matrix_files:

        A_dataframe = pd.read_csv(
            path_to_read + filename, sep='   ', header=None, engine='python')
        A = A_dataframe.values  # we will use a list of numpy arrays, NOT pandas dataframes
        matrices.append(A)# append a matrix to our list
        if "ASD" in filename:
            target_vector.append(1)
        elif "TD" in filename:
            target_vector.append(0)
    asd_dict = {}
    asd_dict['X'] = np.array(matrices)
    asd_dict['y'] = np.array(target_vector)
    if distances:
        dist_matrix_list = []
        for item in distance_files:
            # print(item)
            cur_coord = get_autism_distances(path_to_read + item)
            cur_dist_mtx = get_distance_matrix(cur_coord)
            dist_matrix_list += [cur_dist_mtx]

        asd_dict['dist'] = np.array(dist_matrix_list)

    return asd_dict


### Функция понижения ранга матрицы

In [5]:
def matrix_eig(data, k = 0):
    new_data = {}
    new_data['y'] = data['y']
    new_data['dist'] = data['dist']
    new_data['X'] = np.zeros(shape = (data['X'].shape[0], data['X'].shape[1], data['X'].shape[1] - k))
    for i in np.arange(data['X'].shape[0]):
        curs, vecs = np.linalg.eig(data['X'][i])
        indeces_del = range(curs.size)[(curs.size - k):]
        new_data['X'][i] = np.delete(vecs, indeces_del, axis=1).astype('float')
    return new_data

## Сделаем один пайплайн

In [6]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

In [21]:
data = 'Data/dti/'
data = Transformer(get_autism).fit_transform(data)

In [22]:
data = Transformer(matrix_eig, {'data': data, 'k': 20}).fit_transform(data)
data = Transformer(degrees, collect=['degrees']).fit_transform(data)

In [23]:
print data

(array([[  7.66089425e+00,  -5.71351424e+00,  -1.20033507e+00, ...,
          3.06759167e-03,  -6.90826962e-02,  -2.84482455e-02],
       [ -1.01109909e+01,  -8.48883509e-01,   5.65138661e-01, ...,
          2.50782840e-01,   1.25505953e-01,  -2.48150557e-01],
       [  6.71053975e+00,   7.07528285e+00,  -4.95512366e+00, ...,
         -1.69002764e-02,   3.66884922e-01,  -2.83693075e-01],
       ..., 
       [  6.44225227e+00,   4.61363983e+00,   6.84541856e+00, ...,
         -2.13978371e-01,  -4.54004713e-01,  -1.57328848e-01],
       [  1.05407213e+01,   3.35618199e+00,   2.31842377e+00, ...,
          1.55627170e-01,  -6.99457686e-01,   5.26605955e-01],
       [  7.55466094e+00,   5.45682505e+00,  -2.65237903e+00, ...,
         -5.39388664e-02,  -1.58016074e-01,   3.40319641e-02]]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [24]:
X, y = data 
print X.shape, y.shape

(94, 244) (94,)


In [25]:
steps = [('selector', VarianceThreshold()), ('scaler', MinMaxScaler()), ('classifier', LogisticRegression())] 
pipeline = Pipeline(steps)

In [26]:
param_grid = dict(classifier__penalty=['l1', 'l2'])
scoring = 'roc_auc'
grid_clf = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=scoring, n_jobs=-1, cv=grid_cv)
grid_clf.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=0, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('selector', VarianceThreshold(threshold=0.0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'classifier__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [27]:
steps[-1] = steps[-1][0], grid_clf.best_estimator_
pipeline = Pipeline(steps)
scores = cross_val_score(pipeline, X, y, scoring=scoring, cv=eval_cv, n_jobs=-1)
np.mean(scores), np.std(scores)

(0.61133333333333328, 0.16110865898517063)

### Приведу некоторые результаты для различных К

|       scores       |         std        |  k |
|      :------:      |        :---:       | :-:|
| 0.59233333333333338| 0.12568081264324588|  0 |
| 0.59233333333333338| 0.12568081264324588|  1 | 
| 0.59233333333333338| 0.12568081264324588|  2 | 
| 0.59433333333333338| 0.13016271867679058| 10 |
| 0.61133333333333328| 0.16110865898517063| 20 |
| 0.63233333333333341| 0.16096272860510288| 30 |
| 0.63233333333333341| 0.16096272860510288| 40 |
| 0.63233333333333341| 0.16096272860510288| 45 |
| 0.54400000000000004| 0.27115309328864384| 50 |
| 0.21695468036742913| 0.22945079356294823| 75 |
| 0.49466666666666664| 0.21695468036742913| 100|

## Попробуем сделать это, используя класс Papiliner

Здесь возникают проблемы, из-за того, что работая с один пайплайном мы явно можем задать парамеры функции. Тут же сделать это сложнее.

Есть несколько решений:  
1. Задавать k по дефолту в функции  
2. Задавать параметр data функции matrix_eig через стороннюю переменную, расчитанную ранее

На мой взгляд, лучши решением будет первый вариант. Поэтому на нем я и остановился

In [37]:
def matrix_eig_k(data, k = 20):
    new_data = {}
    new_data['y'] = data['y']
    new_data['dist'] = data['dist']
    new_data['X'] = np.zeros(shape = (data['X'].shape[0], data['X'].shape[1], data['X'].shape[1] - k))
    for i in np.arange(data['X'].shape[0]):
        curs, vecs = np.linalg.eig(data['X'][i])
        indeces_del = range(curs.size)[(curs.size - k):]
        new_data['X'][i] = np.delete(vecs, indeces_del, axis=1).astype('float')
    return new_data

In [40]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

data = [('UCLAsource', Transformer(get_autism)),
        #('UCLAbaseline', Transformer(get_baseline))
       ]

#Only low_rank
weighters = [#('origW', Transformer(orig)),
             #('binar', Transformer(binar_norm)),
             #('wbysqdist', Transformer(wbysqdist)),
             ('low_rank', Transformer(matrix_eig_k))]


normalizers = [('origN', Transformer(orig)),
               ('spectral', Transformer(spectral_norm))]

featurizers = [('origF', Transformer(orig, collect=['X'])),
               ('degrees', Transformer(degrees, collect=['degrees']))]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler()),
           ('origS', FunctionTransformer(orig))]

#For tests, don`t use XGB, it needs a lot of time
classifiers = [('LR', LogisticRegression()),
               ('RF', RandomForestClassifier()),
               ('SVC', SVC()),
               #('XGB', XGBClassifier(nthread=1)),
               ('SGD', SGDClassifier())]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

banned_combos = [#('UCLAsource', 'origN'),
                 ('UCLAsource', 'origF'),
                 ('UCLAbaseline', 'degrees'),
                 ('UCLAbaseline', 'binar'),
                 ('UCLAbaseline', 'wbysqdist'),
                 ('UCLAbaseline', 'spectral'),
                 ('UCLAbaseline', 'low_rank'),
                 ('spectral', 'low_rank'),
                 ('LR', 'origS'),
                 ('SVC', 'origS'),
                 ('SGD', 'origS'),
                 ('RF', 'minmax'),
                 ('XGB', 'minmax')]

param_grid = dict(
    LR=dict(
        C=[0.01, 0.05, 0.1] + [0.05*i for i in range(3, 21)],
        max_iter=[50, 100, 500],
        penalty=['l1', 'l2']
    ),
    SGD=dict(
        alpha=[0.001, 0.01, 0.1, 0.5, 1.0],
        l1_ratio=[0, 0.2, 0.4, 0.6, 0.8, 1],
        loss=['hinge', 'log', 'modified_huber'],
        n_iter=[50, 100, 200],
        penalty=['elasticnet']
    ),
    SVC=dict(
        C=[0.0005, 0.001, 0.005, 0.01] + [i*0.05 for i in range(1,11)],
        degree=[2, 3, 4],
        kernel=['linear', 'poly', 'rbf', 'sigmoid'],
        max_iter=[50, 100, 150],
    ),
    RF=dict(
        criterion=['entropy', 'gini'],
        max_depth=[3, 5, 7, 10, 20],
        max_features=['log2', 'sqrt'] + [0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5, 1.0],
        n_estimators=[10, 50, 100, 200, 500]
    ),
    XGB=dict(
        colsample_bytree=[0.01] + [0.05*i for i in range(1,21)],
        learning_rate=[0.01*i for i in range(1,6)] + [0.05*i for i in range(2,11)],
        max_depth=[i for i in range(1,12)],
        n_estimators=[10, 50, 100, 200, 500],
        nthread=[1],
        reg_alpha=[0, 1],
        reg_lambda=[0, 1],
        subsample=[0.5, 0.7, 1]
    )
)

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,low_rank,origN,degrees,var_threshold,minmax,LR
1,UCLAsource,low_rank,origN,degrees,var_threshold,minmax,SVC
2,UCLAsource,low_rank,origN,degrees,var_threshold,minmax,SGD
3,UCLAsource,low_rank,origN,degrees,var_threshold,origS,RF


In [41]:
pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'], scoring=['roc_auc'])

Removed previous results file -- results.csv.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4


,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,UCLAsource,low_rank,origN,degrees,var_threshold,minmax,LR,0.674645,0.173122,"{'penalty': 'l1', 'C': 0.25, 'max_iter': 50}",0.682,0.144969,[ 0.4 0.8 0.92 0.7 0.6 0.7 0.5 0....
1,UCLAsource,low_rank,origN,degrees,var_threshold,minmax,SVC,0.62039,0.194252,"{'kernel': 'poly', 'C': 0.45, 'max_iter': 50, ...",0.549,0.231838,[ 0.3 0.96 0.68 0.2 0.65 0.5 0.85 0....
2,UCLAsource,low_rank,origN,degrees,var_threshold,minmax,SGD,0.709397,0.169483,"{'penalty': 'elasticnet', 'loss': 'modified_hu...",0.656333,0.148507,[ 0.43333333 0.76 0.52 0.4 ...
3,UCLAsource,low_rank,origN,degrees,var_threshold,origS,RF,0.758511,0.129327,"{'n_estimators': 50, 'max_features': 0.01, 'cr...",0.463,0.221181,[ 0. 0.4 0.48 0.7 0.65 0.15 0.4 0....
